# Echo State Network against Time Series

In [ ]:
import time

import torch
from torch.utils.data import TensorDataset, DataLoader
from dataclasses import dataclass, field
import matplotlib.pyplot as plt
import numpy as np

# from qbraid_algorithms import EchoStateNetwork, EchoStateReservoir

In [ ]:
@dataclass
class EchoStateReservoir:  # pylint: disable=too-many-instance-attributes
    """Dataclass for a reservoir component of an Echo State Network."""

    input_size: int
    hidden_size: int
    sparsity: float = 0.9
    spectral_radius: float = 0.99
    a: float = 0.6
    leak: float = 1.0
    max_retries: int = 3
    w_in: torch.Tensor = field(init=False)
    w: torch.Tensor = field(init=False)
    x: torch.Tensor = field(init=False)

    def __post_init__(self):
        self.w_in = self._generate_w_in()
        self.w = self._generate_w(max_retries=self.max_retries)
        self.x = torch.zeros(self.hidden_size, 1)

    def _generate_w_in(self, mean: float = 0.0) -> torch.Tensor:
        """Generates and returns a random input weight matrix, w_in."""
        return torch.randn(self.hidden_size, self.input_size + 1).normal_(mean=mean, std=self.a)

    def _generate_w(self, mean: float = 0.0, max_retries: int = 3) -> torch.Tensor:
        """Generates a sparse internal weight matrix, w, with retries if necessary."""
        for attempt in range(max_retries + 1):
            w = torch.randn(self.hidden_size, self.hidden_size).normal_(
                mean=mean, std=self.spectral_radius
            )
            w = torch.where(torch.rand_like(w) > self.sparsity, w, torch.zeros_like(w))
            eigenvalues = torch.linalg.eigvals(w)  # pylint: disable=not-callable
            max_abs_eigenvalue = torch.max(torch.abs(eigenvalues)).item()
            if max_abs_eigenvalue != 0:
                w *= self.spectral_radius / max_abs_eigenvalue
                return w
            if attempt == max_retries:
                raise ReservoirGenerationError(
                    f"Failed to generate a valid weight matrix after {max_retries} retries; "
                    "max eigenvalue was zero."
                )
        return w

    def evolve(self, u: torch.Tensor) -> None:
        """
        Updates and returns the state representation x for a given input u.

        Args:
            u: Input vector.

        """
        temp_state = torch.tanh(
            torch.mm(self.w_in, torch.cat((torch.tensor([[1.0]]), u), 0)) + torch.mm(self.w, self.x)
        )
        new_state = (1 - self.leak) * self.x + self.leak * temp_state
        self.x = new_state

In [ ]:
class EchoStateNetwork(torch.nn.Module):
    """
    An Echo State Network module that combines a Reservoir with a fully connected output layer.

    Attributes:
        reservoir (Reservoir): The reservoir component of the ESN.
        fc (torch.nn.Linear): Linear layer to map reservoir states to output dimensions.
        softmax (torch.nn.Softmax): Softmax activation function for generating output probabilities.
    """

    def __init__(self, reservoir: EchoStateReservoir, output_size: int):
        """
        Initializes the Echo State Network with a reservoir and a linear output layer.

        Args:
            reservoir (EchoStateReservoir): Reservoir component of the ESN.
            output_size (int): Size of each output sample.
        """
        super().__init__()
        self.reservoir = reservoir
        self.fc = torch.nn.Linear(in_features=reservoir.hidden_size, out_features=output_size)
        self.softmax = torch.nn.Softmax(dim=-1)

    def forward(self, input_data: torch.Tensor) -> torch.Tensor:
        """
        Processes the input through the network and returns the output probabilities.

        Args:
            input_data: A tensor of input data.

        Returns:
            torch.Tensor: A tensor containing the softmax probabilities of the outputs.
        """
        u = input_data.flatten()  # Flatten data into a single vector
        u = input_data.unsqueeze(dim=0).t()  # Transpose to desired input shape for the reservoir

        self.reservoir.evolve(u)  # Pass through reservoir

        h = self.fc(self.reservoir.x.t())  # Pass transposed output x through the linear layer
        # h = self.softmax(h)  # Apply softmax to get probabilities
        return h # Softmax unnecessary for 1-d output

Format Time Series data to work with the ESN

In [ ]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        seq_x, seq_y = data[i:i+n_steps], data[i+n_steps]
        X.append(seq_x)
        y.append(seq_y)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32).view(-1,1, 1)

n_steps = 10

t = np.linspace(0, 2 * np.pi * 10, 1000)
data = np.sin(t)
X, y = create_sequences(data, n_steps)
trainset = TensorDataset(X, y)
trainloader = DataLoader(trainset, shuffle=True)

Initialize ESN, optimizer, and loss criterion

In [ ]:
input_size = 10
output_size = 1
hyperparams = {
    "hidden_size": 2500,
    "sparsity": 0.9,
    "spectral_radius": 0.99,
    "a": 0.6,
    "leak": 1.0,
}

reservoir = EchoStateReservoir(input_size, **hyperparams)
esn = EchoStateNetwork(reservoir, output_size).float()

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(esn.parameters(), lr=0.00001)
len_data = len(trainset)
nsamples = 1000
nepochs = 200

Train the network

In [ ]:
loss_values = []
start = time.time()
for epoch in range(nepochs):
    total_loss = 0.0
    for i, dat in enumerate(trainset, 0):
        seq = dat[0]
        target = dat[1] 

        optimizer.zero_grad()

        output = esn(seq)[0]  
        loss = criterion(output, target) 

        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    average_loss = total_loss / nsamples
    loss_values.append(average_loss)
    print(f"Epoch {epoch+1}, Loss: {average_loss}")

end = time.time()
print("Training complete")
print("num samples = " + str(nsamples))
print("num epochs = " + str(nepochs))
print("total time = " + str(end - start) + " sec")


Plot the loss

In [ ]:
fig, ax = plt.subplots(dpi=100)
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
ax.plot(range(1, nepochs + 1), loss_values, color="red")
plt.show()

Plot against desired data, giving R^2 as well

In [ ]:
def test():
    t_test = np.linspace(2 * np.pi * 10, 2 * np.pi * 12, 400)  # continuing from where training data stopped
    data_test = np.sin(t_test)
    X_test, y_test = create_sequences(data_test, n_steps)
    testset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(testset, shuffle=True)
    esn.eval()
    predictions = []
    with torch.no_grad():
        for inputs, targets in test_loader.dataset:
            output = esn(inputs)[0]
            predictions.append(output)

    return predictions


predictions = test()
t_test = np.linspace(2 * np.pi * 10, 2 * np.pi * 12, 400)
plt.plot(torch.cat(predictions).numpy(), label="Predictions")
plt.plot([np.sin(y) for y in t_test], label="True values")
plt.legend()

y_true = np.sin(t_test)
y_pred = torch.cat(predictions).numpy().flatten()
y_true_mean = np.mean(y_true)
ss_tot = np.sum((y_true - y_true_mean) ** 2)
ss_res = np.sum((y_true[:390] - y_pred) ** 2)
r2 = 1 - ss_res / ss_tot
print("R^2: ", r2)
